In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2

In [ ]:
import quick_pp.las_handler as las

well_name = '15-9-F-11-T2'
with open(rf'data\01_raw\VOLVE\{well_name}.las', 'rb') as f:
    df, header = las.read_las_files([f])

df['PERM'] = df.KLOGH
df['PHIT'] = df.PHIF
df['SWT'] = df.SW
df['VCLD'] = df.VSH * (1 - df.PHIT)
df['VSAND'] = (1 - df.VSH) * (1 - df.PHIT)
df['VHC'] = df.PHIT * (1 - df.SWT)
df['DTC'] = df.DT

In [ ]:
from quick_pp.plotter.well_log import plotly_log

fig = plotly_log(df, well_name=well_name, depth_uom='m')
fig.show(config=dict(scrollZoom=True))

In [ ]:
from quick_pp.rock_physics.geomechanics import *

df_copy = df.copy()
df_copy['RHOB'] = df_copy['RHOB'].clip(1.8, 3)

df_copy['Vp'] = estimate_compressional_velocity(df_copy['RHOB'])
df_copy['VP'] = 1 / df_copy['DT'] * ( 1e6 / 3.281)

df_copy.plot(x='DEPTH', y=['Vp', 'VP'])

In [ ]:
df_copy['Rhob'] = estimate_gardner_density(1 / df_copy['DT'] * (1e6 / 3.281))

df_copy.plot(x='DEPTH', y=['Rhob', 'RHOB'], ylim=(1.8, 2.85))

In [ ]:
from quick_pp.rock_physics.rpm import *

vp = 1 / df_copy['DT'] * (1e6 / 3.281)
if 'DTS' in df_copy.columns:
    vs= 1 / df_copy['DTS'] * (1e6 / 3.281)
else:
    vs = estimate_shear_velocity(vp)
rhob = df_copy['RHOB'].clip(1, 3)
fig = qaqc_xplots(rhob, vp, vs)

In [ ]:
fig = fluid_typing_xplots(rhob, vp, vs)

In [ ]:
from quick_pp.rock_physics.rpm import *

fig = QI_screening(df_copy['GR'], df_copy['RHOB'], vp)

In [ ]:
from quick_pp.rock_physics.rpm import *

fig = elastic_bounds_plot(df_copy['RHOB'], vp, vs, df_copy.PHIT)

In [ ]:
from quick_pp.rock_physics.rpm import *

fig = rpt_plot(
    df_copy['RHOB'], vp, vs, model='stiff_sand', fluid_type='oil',
    Cn=None, f=0, sigma=20
)